In [1]:
from compute_lda import *

/home/raymond/anaconda3/envs/topicModeling/lib/python3.7/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


### Some functions

In [2]:
def count_country(df,name):
    cols = ['port_of_unlading','foreign_port_of_lading','place_of_receipt'
            ,'port_of_destination','foreign_port_of_destination'
           ,'consignee_address','shipper_address']
    print('total rows is {}'.format(data.shape[0]))
    for i,col in enumerate(cols):
        temp_col = data[col].str.contains(name,case=False,regex=False)
        print('{} contains {} of rows of {}'.format(col,temp_col.sum(),name))
        if i==0:
            sum_col = temp_col.copy()
        else:
            sum_col = sum_col.add(temp_col,fill_value=0)
    print('{} of rows are {}'.format(sum_col[sum_col>0].sum(),name))

In [3]:
six_countries = ['Germany','Belgium', 'Vietnam' ,'Spain', 'United Kingdom','France']

### Data Processing

In [4]:
data = read_data()

In [5]:
data.shape

(3825317, 26)

In [6]:
def simple_process_data(data):
    # Basic data cleaning
    data = data.dropna(subset=['shipper_party_name','harmonized_number'])
    replace_char = ",.+=_-><\'\":;()!?~/\\@#$%^&*~`[]{}"
    replace_dict = {key:value for (key,value) in zip(replace_char,itertools.repeat(''))}
    data['cl_shipper_party_name'] = data['shipper_party_name'].str.translate(str.maketrans(replace_dict)).copy()
    data = data.assign(shipper_id=(data['cl_shipper_party_name']).astype('category').cat.codes)
    data['6_harmonized_number'] = data['harmonized_number'].apply(lambda x: str(x)[0:6])
    return data

In [7]:
data = simple_process_data(data)

/home/raymond/anaconda3/envs/topicModeling/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
data.shape

(1315991, 29)

In [9]:
data.columns

Index(['identifier', 'port_of_unlading', 'estimated_arrival_date',
       'foreign_port_of_lading', 'record_status_indicator', 'place_of_receipt',
       'port_of_destination', 'foreign_port_of_destination',
       'actual_arrival_date', 'consignee_name', 'consignee_address',
       'consignee_contact_name', 'consignee_comm_number_qualifier',
       'consignee_comm_number', 'shipper_party_name', 'shipper_address',
       'shipper_contact_name', 'shipper_comm_number_qualifier',
       'shipper_comm_number', 'description_sequence_number', 'piece_count',
       'description_text', 'harmonized_number', 'harmonized_value',
       'harmonized_weight', 'harmonized_weight_unit', 'cl_shipper_party_name',
       'shipper_id', '6_harmonized_number'],
      dtype='object')

### Bag of Words

In [11]:
bag_of_words = data.groupby(by=['6_harmonized_number','shipper_party_name']).size().unstack(fill_value=0)

In [14]:
bag_of_words.sum().describe()

count    39638.000000
mean        33.200237
std        292.931290
min          1.000000
25%          1.000000
50%          4.000000
75%         12.000000
max      26376.000000
dtype: float64

In [15]:
bag_of_words.sum().sort_values()

shipper_party_name
GOTEX SA                                   1
SOL & LUNA-CRIAMENDI ESPANA SL             1
SOL & LUNA CRIAMENDI ESPA A SL             1
PRIME CARGO POLAND                         1
COPREX,S.L.                                1
COPRIMA SL                                 1
COPRIMA, S.L.                              1
GEODIS CALBERSON BOSPHORUS                 1
GEODIS BERNIS LIMOGES                      1
SOL Y LUNA CRIAMENDI ESPANA SL             1
CORDEE PUBLICATIONS                        1
SOKOA                                      1
KORMANN ROCKSTER RECYCLER GMBH             1
GEODIS BERNIS - LIMOGES                    1
SOILMEC AUSTRALIA P L                      1
SOIL MACHINE DYNAMICS (SMD)                1
COREY BRADFORD SLATE                       1
SOGU APLICACIONES SL                       1
SOGEVINUS FINE WINES SA                    1
CORDENET PRODUTORA DE REDES AGRIC          1
NANOBITS LIMITED                           1
MR EVEN NATHANIEL WALSER,           